# Better Basket Technical Assessment
#### Read JSONs and Initialize MATCH_SCORE_THRESHOLD (more on this later)

In [ ]:
import json
MATCH_SCORE_THRESHOLD = 70.0
with open("grocery_store_a.json", 'r', encoding="utf-8") as file:
    grocery_store_a = json.load(file)
with open("grocery_store_b.json", 'r', encoding="utf-8") as file:
    grocery_store_b = json.load(file)

### Use Beautiful Soup to scrape Grocery Store B's Products
Specifically getting the name, price, amount, and quantity bought. Would use Selenium if this was on the web.

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
products = []
for i in range(len(grocery_store_b)):
    soup = BeautifulSoup(grocery_store_b[i]['data']['html_data'], "html.parser")
    for item in soup.find_all("div", class_="product-grid-item"):
        name = item.find("h3").text.strip() if item.find("h3") else None
        price = item.find("p", class_="precio").text.strip() if item.find("p", class_="precio") else None
        amount = item.find("p", class_="text-center text-muted").text.strip() if item.find("p", class_="text-center text-muted") else None
        quantity = item.find("input", {"name": "qty"})["value"] if item.find("input", {"name": "qty"}) else None
        
        products.append([name, price, amount, quantity])
grocery_store_b_df = pd.DataFrame(products, columns=["Name", "Price", "Amount", "Quantity"])

print(grocery_store_b_df)

                                      Name   Price   Amount Quantity
0                    SPRITE ZERO SUGAR 8PK   $4.75    10 OZ        1
1           PAN PEPIN HOT DOG INTEGRAL 8PK   $3.19  12.5 OZ        1
2            TORTILLA ESPANOLA CON CEBOLLA   $6.19  17.6 OZ        1
3          CAMPBELLS CHUNKY CHICKEN NOODLE   $4.15  18.6 OZ        1
4       KELLOGGS FROSTED FLAKES CEREAL CUP   $1.75   2.1 OZ        1
...                                    ...     ...      ...      ...
12797  PLEDGE MULTI SURFACE TRIGGER CITRUS   $6.29    25 OZ        1
12798         DEPEND UNDERWEAR SM/MED MALE  $29.99    44 CT        1
12799               LISBOA SALMON PORTIONS  $13.29     2 LB        1
12800           GOYA NECTAR PINA & GUAYABA   $1.99  33.8 OZ        1
12801       BORDEN MAGNOLIA CONDENSED MILK   $3.15    14 OZ        1

[12802 rows x 4 columns]


Checking how many rows have quanitity greater than 1. This number 295 ends up being the same as the number of rows with the #/$#.# format.

In [18]:
# Convert the 'Quantity' column to numeric, forcing errors to NaN (if any)
grocery_store_b_df['Quantity'] = pd.to_numeric(grocery_store_b_df['Quantity'], errors='coerce')

# Count the number of rows where Quantity > 1
count_quantity_greater_than_1 = grocery_store_b_df[grocery_store_b_df['Quantity'] > 1].shape[0]

print(f"Number of rows where Quantity > 1: {count_quantity_greater_than_1}")


Number of rows where Quantity > 1: 295


### Price Normalization of Grocery Store B
Goal: Have price be represented as a float value \
How? TL;DR: Rejex to match each format then remove unnecessary pieces to form the float

Case 1: $#.#, the easy case. Remove the dollar sign and convert it to a float.

In [ ]:
# PRICE NORMALIZATION
import re
def check_price_format(price):
    return bool(re.match(r'^\$\d+(\.\d{1,2})?$', price))

# Identify rows where the price does not match the expected format
incorrect_prices = grocery_store_b_df[~grocery_store_b_df['Price'].apply(check_price_format)]

# Print rows where the price does not follow the "$" format
print("Rows with incorrectly formatted prices:")
print(incorrect_prices)
incorrect_prices.to_csv("incorrect_prices.csv")
# Identify valid rows where the price follows the correct format
valid_rows = grocery_store_b_df['Price'].apply(check_price_format)

# Convert only valid price rows to float by removing the "$" sign
grocery_store_b_df.loc[valid_rows, 'Price'] = grocery_store_b_df.loc[valid_rows, 'Price'].replace(r'^\$', '', regex=True).astype(float)
grocery_store_b_df = grocery_store_b_df[valid_rows]
print(grocery_store_b_df)

Rows with incorrectly formatted prices:
                                    Name     Price    Amount  Quantity
17         BACALAO FILETE SIN ESPINAS EU  $9.59 LB  POR PESO         1
21             KRAFT EXTRA SHARP CHEDDAR   2/$5.00      8 OZ         2
22                        GATORADE GRAPE   5/$5.00   33.8 OZ         5
24                        SUPERMAX ELBOW       99¢   14.1 OZ         1
36                 LECHUGA ICEBERG CELLO   2/$3.00      1 EA         2
...                                  ...       ...       ...       ...
12653  SIERRA CORTE SELECTO CONG ECUADOR  $5.49 LB  POR PESO         1
12695  PAVO CONGELADO GRADO A 16-22LB US  $2.79 LB  POR PESO         1
12731    DORADO REBANADO CONG DE ECUADOR  $3.99 LB  POR PESO         1
12737                      PEPSI REGULAR       75¢   1.25 LT         1
12752                        7UP REGULAR       75¢   1.25 LT         1

[833 rows x 4 columns]
                                      Name  Price   Amount  Quantity
0              

Case 2: #¢; Remove the cent and divide by 100. \

 Keep all remaining cases that don't match the rejex in the incorrect_prices dataframe and put corrected versions into their own new one. We will concatenate all the corrected dataframes in the end.

In [21]:
import re

def check_cent_format(price):
    return bool(re.match(r'^\d{1,2}¢$', price))

# Apply the format check to filter correct cent prices
cent_prices = incorrect_prices[incorrect_prices['Price'].apply(check_cent_format)]

# Remove the cent sign and convert to float by dividing by 100
cent_prices['Price'] = cent_prices['Price'].apply(lambda x: float(x.replace('¢', '')) / 100)

# Get the remaining rows that do not match the cent format
incorrect_prices = incorrect_prices[~incorrect_prices['Price'].apply(check_cent_format)]

cent_prices, incorrect_prices


C:\Users\richa\AppData\Local\Temp\ipykernel_33660\2086008963.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cent_prices['Price'] = cent_prices['Price'].apply(lambda x: float(x.replace('¢', '')) / 100)


(                                          Name  Price   Amount  Quantity
 24                              SUPERMAX ELBOW   0.99  14.1 OZ         1
 40                                 7UP REGULAR   0.95  1.75 LT         1
 62        LIBBYS VIENNA SAUSAGE CHICKEN & PORK   0.83   4.6 OZ         1
 109     GOYA HABICHUELAS ROSADAS EN AGUA Y SAL   0.93  10.5 OZ         1
 118         MARUCHAN RAMEN NOODLE SOUP CHICKEN   0.59     3 OZ         1
 ...                                        ...    ...      ...       ...
 12226                   BAUDUCCO WAFER COCONUT   0.95     5 OZ         1
 12456                         TOP RAMEN SHRIMP   0.85     3 OZ         1
 12464  PETIT NATURAL DEFENSE ZANAHORIA NARANJA   0.59  11.2 OZ         1
 12737                            PEPSI REGULAR   0.75  1.25 LT         1
 12752                              7UP REGULAR   0.75  1.25 LT         1
 
 [216 rows x 4 columns],
                                            Name      Price    Amount  Quantity
 17  

Case 3: $#.# LB; Remove the dollar sign, LB, trim it and convert it to float.

In [23]:
def check_LB_format(price):
    return bool(re.match(r'^\$\d+(\.\d{1,2})?\s*LB$', price))

# Apply the format check to filter correct cent prices
lb_prices = incorrect_prices[incorrect_prices['Price'].apply(check_LB_format)]

# Remove the cent sign and convert to float by dividing by 100
# Update the 'Price' column to reflect the actual price (convert cents to dollars)
lb_prices['Price'] = lb_prices['Price'].apply(lambda x: float(x.replace('$', '').replace(' LB', '')))

# Update the 'Amount' column to "1lb" for all rows in lb_prices
lb_prices['Amount'] = '16 OZ'
incorrect_prices = incorrect_prices[~incorrect_prices['Price'].apply(check_LB_format)]

print(lb_prices, incorrect_prices)

                                               Name  Price Amount  Quantity
17                    BACALAO FILETE SIN ESPINAS EU   9.59  16 OZ         1
38                                     MANZANA GALA   2.27  16 OZ         1
42                                       PERA VERDE   2.27  16 OZ         1
48     COSTILLAS COUNTRY STYLE FRESCA. US CON HUESO   3.19  16 OZ         1
49          MASA MOLIDA DE RES CAB FRESCA CHOICE EU   6.79  16 OZ         1
...                                             ...    ...    ...       ...
12541                        BH CUBE CHIPOTLE GOUDA  11.59  16 OZ         1
12638      SIERRA REBANADA CORTE MIXTO CONG ECUADOR   4.99  16 OZ         1
12653             SIERRA CORTE SELECTO CONG ECUADOR   5.49  16 OZ         1
12695             PAVO CONGELADO GRADO A 16-22LB US   2.79  16 OZ         1
12731               DORADO REBANADO CONG DE ECUADOR   3.99  16 OZ         1

[314 rows x 4 columns]                              Name    Price    Amount  Quantity
2

C:\Users\richa\AppData\Local\Temp\ipykernel_33660\510796585.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lb_prices['Price'] = lb_prices['Price'].apply(lambda x: float(x.replace('$', '').replace(' LB', '')))
C:\Users\richa\AppData\Local\Temp\ipykernel_33660\510796585.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lb_prices['Amount'] = '16 OZ'


Case 4: #/$#.#; Get the first number which represents the num items, divide the price by num items and set quantity to 1 to emulate the price of buying 1 item

In [ ]:
import re
import pandas as pd

# Define the updated regex pattern
pattern = r"^\s*(\d+)\s*\/\s*\$(\d+(\.\d{2})?)\s*$"

# Function to check the condition
def check_price(row):
    match = re.match(pattern, row["Price"])
    if match:
        first_number = int(match.group(1))  # The first number in the price
        return first_number == row["Quantity"]
    return False

# Apply the function to check each row
incorrect_prices['is_match'] = incorrect_prices.apply(check_price, axis=1)
# Filter rows where is_match is True and False
division_prices = incorrect_prices[incorrect_prices['is_match'] == True]
incorrect_prices = incorrect_prices[incorrect_prices['is_match'] == False]
# Extracting the price before the '/' and converting it to a float
division_prices['num_items'] = division_prices['Price'].apply(lambda x: float(x.split('/')[0].replace('$','').replace(',','')))

# Extracting the price after the '$' sign and converting it to a float
division_prices['Price'] = division_prices['Price'].apply(lambda x: float(x.split('$')[1].replace(',','')))
# Divide Amount by num_items
division_prices['Amount'] = division_prices.apply(lambda row: f"{float(row['Amount'].split()[0]) / row['num_items']:.2f} {row['Amount'].split()[1]}", axis=1)

# Set all quantities to 1
division_prices['Quantity'] = 1

# Update the price by dividing it by num_items
division_prices['Price'] = division_prices['Price'] / division_prices['num_items']

# Drop the num_items column
division_prices = division_prices.drop(columns=['num_items'])
division_prices



                                 Name    Price   Amount  Quantity  is_match
21          KRAFT EXTRA SHARP CHEDDAR  2/$5.00     8 OZ         2      True
22                     GATORADE GRAPE  5/$5.00  33.8 OZ         5      True
36              LECHUGA ICEBERG CELLO  2/$3.00     1 EA         2      True
39         MOTTS APPLE JUICE ORIGINAL  2/$6.00    64 OZ         2      True
45     KRAFT VELVEETA SLICES ORIGINAL  2/$5.00    12 OZ         2      True
...                               ...      ...      ...       ...       ...
12395         MINUTE MAID APPLE JUICE  4/$5.00    12 OZ         4      True
12409        MINUTE MAID ORANGE JUICE  4/$5.00    12 OZ         4      True
12422      MINUTE MAID TROPICAL BLEND  4/$5.00    12 OZ         4      True
12499         PEPSI DIET MINI CAN 6PK  2/$5.00   7.5 OZ         2      True
12514        PEPSI BLACK MINI CAN 6PK  2/$5.00   7.5 OZ         2      True

[295 rows x 5 columns]


C:\Users\richa\AppData\Local\Temp\ipykernel_33660\827262143.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incorrect_prices['is_match'] = incorrect_prices.apply(check_price, axis=1)


Case 5: #¢ LB, remove the cent sign, change the Amount to 16 OZ.

In [31]:
import re

def check_cent_LB_format(price):
    return bool(re.match(r'^\d+¢\s*LB$', price))

# Apply the format check to filter correct cent prices
cent_lb_prices = incorrect_prices[incorrect_prices['Price'].apply(check_cent_LB_format)]

# Remove the cent symbol and convert to float (divide by 100 to convert cents to dollars)
cent_lb_prices['Price'] = cent_lb_prices['Price'].apply(lambda x: float(x.replace('¢', '').replace(' LB', '')) / 100)

# Update the 'Amount' column to "1lb" for all rows in cent_lb_prices
cent_lb_prices['Amount'] = '16 OZ'

# Filter out the rows with 'cent LB' format from incorrect_prices
incorrect_prices = incorrect_prices[~incorrect_prices['Price'].apply(check_cent_LB_format)]

print(cent_lb_prices)
print(incorrect_prices)


                                  Name  Price Amount  Quantity  is_match
37                  CALABAZA EN PEDAZO   0.47  16 OZ         1     False
54               BATATA FRESCA SELECTA   0.77  16 OZ         1     False
226                      BATATA BLANCA   0.77  16 OZ         1     False
332        REMOLACHA FRESCA COSTA RICA   0.97  16 OZ         1     False
733    MUSLOS DE POLLO FRESCOS F/P. EU   0.79  16 OZ         1     False
1319                   CALABAZA ENTERA   0.37  16 OZ         1     False
8144                     REPOLLO VERDE   0.87  16 OZ         1     False
12284  CADERAS DE POLLO FRESCAS F/P EU   0.79  16 OZ         1     False
Empty DataFrame
Columns: [Name, Price, Amount, Quantity, is_match]
Index: []


Concat all the normalized DFs

In [32]:
full_grocery_b_df = pd.concat([grocery_store_b_df,division_prices, lb_prices, cent_prices, cent_lb_prices])
full_grocery_b_df

,Name,Price,Amount,Quantity,is_match
0,SPRITE ZERO SUGAR 8PK,4.75,10 OZ,1,NaN
1,PAN PEPIN HOT DOG INTEGRAL 8PK,3.19,12.5 OZ,1,NaN
2,TORTILLA ESPANOLA CON CEBOLLA,6.19,17.6 OZ,1,NaN
3,CAMPBELLS CHUNKY CHICKEN NOODLE,4.15,18.6 OZ,1,NaN
4,KELLOGGS FROSTED FLAKES CEREAL CUP,1.75,2.1 OZ,1,NaN
...,...,...,...,...,...
332,REMOLACHA FRESCA COSTA RICA,0.97,16 OZ,1,False
733,MUSLOS DE POLLO FRESCOS F/P. EU,0.79,16 OZ,1,False
1319,CALABAZA ENTERA,0.37,16 OZ,1,False
8144,REPOLLO VERDE,0.87,16 OZ,1,False


### Add the Unit Price as the Price / Amount sold by

In [33]:
import re
import pandas as pd

# Clean Amount to handle cases like '11.3OZ', '28.00/OZ', and '28.00 OZ'
def clean_amount(amount):
    # Handle case like '11.3OZ' (without space or slash)
    match = re.match(r'(\d+(\.\d+)?)\s*(\D+)', amount)
    if match:
        return match.group(1)  # Return the numeric part as string (no unit)
    # Handle case like '28.00/OZ' by taking the first part before '/'
    elif '/' in amount:
        return amount.split('/')[0]
    else:
        return amount.split()[0]  # If space-separated, take the first part (numeric value)

# Apply the cleaning function to extract the numeric value and the unit
full_grocery_b_df['Amount_Value'] = full_grocery_b_df['Amount'].apply(lambda x: float(clean_amount(x)))
full_grocery_b_df['Unit'] = full_grocery_b_df['Amount'].apply(lambda x: re.sub(r'[\d/\.]', '', x))  # Extract unit

# Calculate the new column for Amount / Price
full_grocery_b_df['UnitPrice'] = full_grocery_b_df['Price'] / full_grocery_b_df['Amount_Value']

# Display the updated DataFrame
print(full_grocery_b_df)


                                     Name Price   Amount  Quantity is_match  \
0                   SPRITE ZERO SUGAR 8PK  4.75    10 OZ         1      NaN   
1          PAN PEPIN HOT DOG INTEGRAL 8PK  3.19  12.5 OZ         1      NaN   
2           TORTILLA ESPANOLA CON CEBOLLA  6.19  17.6 OZ         1      NaN   
3         CAMPBELLS CHUNKY CHICKEN NOODLE  4.15  18.6 OZ         1      NaN   
4      KELLOGGS FROSTED FLAKES CEREAL CUP  1.75   2.1 OZ         1      NaN   
...                                   ...   ...      ...       ...      ...   
332           REMOLACHA FRESCA COSTA RICA  0.97    16 OZ         1    False   
733       MUSLOS DE POLLO FRESCOS F/P. EU  0.79    16 OZ         1    False   
1319                      CALABAZA ENTERA  0.37    16 OZ         1    False   
8144                        REPOLLO VERDE  0.87    16 OZ         1    False   
12284     CADERAS DE POLLO FRESCAS F/P EU  0.79    16 OZ         1    False   

       Amount_Value Unit UnitPrice  
0             

In [34]:
# Get all unique values in the 'Unit' column
unique_units = full_grocery_b_df['Unit'].unique()

# Print the unique values
print(unique_units)


[' OZ' ' LB' ' RO' ' CT' ' FT' ' EA' ' LT' ' DZ' ' SB' ' PK' ' GRM' ' ML'
 ' QT' 'OZ' ' CP' ' GL' ' YD' ' PCS' ' TB' ' FO' ' CM' 'PG' ' IN' ' UN'
 ' GC']


## Parse through Grocery Store A retrieving Name, Price, Unit Price, and UnitPriceString

In [174]:
products = []
for i in range(len(grocery_store_a)):
    name = grocery_store_a[i]['data']['product']['name']
    if grocery_store_a[i]['data']["product"]["priceInfo"]["currentPrice"] == None:
        price = None
    else: 
        price = grocery_store_a[i]['data']["product"]["priceInfo"]["currentPrice"]["price"]
    if grocery_store_a[i]['data']["product"]["priceInfo"]["unitPrice"] == None:
        unitPrice = None
        unitPriceString = None
    else:
        unitPrice = grocery_store_a[i]['data']["product"]["priceInfo"]["unitPrice"]["price"]
        unitPriceString = grocery_store_a[i]['data']["product"]["priceInfo"]["unitPrice"]["priceString"]
    products.append([name, price, unitPrice, unitPriceString])

grocery_store_a_df = pd.DataFrame(products, columns=["Name", "Price", "UnitPrice", "UnitPriceString"])
print(grocery_store_a_df)


                                                   Name  Price  UnitPrice  \
0     Pan Pepin, Finest Hawaiian Burger, 12 oz, 8 Count   2.68      0.223   
1       Dove Gentle Exfoliating Beauty Bar, 4 oz, 6 Bar   7.47      0.332   
2     Line 39 Cabernet Sauvignon Red Wine, Californi...  12.98      0.511   
3              Pan Pepin, Hot Dog Buns, 10ct, 15 Ounces   2.74      0.183   
4     Borden Spreadable Butter with Canola Oil, 8 oz...   3.97      0.496   
...                                                 ...    ...        ...   
4995  Sheba Perfect Portions Wet Cat Food Variety Pa...  27.25      0.430   
4996  Orville Redenbacher's Popping & Topping Butter...  28.02        NaN   
4997  Heat Lamp Bulb For Reptile 50w 110v Tortoise W...   3.59        NaN   
4998  School Zone Beginning Reading Grades 1-2 Workb...  10.19        NaN   
4999  Shamrock Gumball Machine St. Patrick's Day Rec...  12.99        NaN   

     UnitPriceString  
0      22.3 ¢/ounces  
1          33.2 ¢/oz  
2     

If there is no price, we remove it from the data and we get the Unit of the product via the Unit Price String

In [ ]:
# Price normalization
grocery_store_a_df = grocery_store_a_df.dropna(subset=['Price'])
grocery_store_a_df['Unit'] = grocery_store_a_df['UnitPriceString'].apply(
    lambda x: x.split("/")[1].capitalize() if x and len(x.split()) > 1 else None
)

In [176]:
grocery_store_a_df["Unit"].unique()

array(['Ounces', 'Oz', 'Fl oz', None, 'G', 'Ea', '100 ct', 'Fluid ounces',
       'Sq ft', 'Count', 'Lb', 'Per 100 count', 'Ml', 'Ft'], dtype=object)

## Name Normalization \
### Goal: Find products that are similar/the same across Grocery Store A & Grocery Store B \
How? RapidFuzz. Gives a score of how similar 2 strings are. Decided to use their partial_ratio function since it compares the best matching subsequence which is useful when strings have similar parts but include extra words. \
Ideally would try to use Machine Learning Technique here to finetune a model to figure out how similar 2 strings are in this context but could not find a dataset.

In [ ]:
import pandas as pd
from rapidfuzz import fuzz
from tqdm import tqdm  # Import tqdm

# Assuming you have the DataFrames grocery_store_a_df and grocery_store_b_df

# Helper function to find the best match row from grocery_store_b_df
def find_best_match(row_a, grocery_store_b_df):
    best_match = None
    best_score = 0
    for _, row_b in grocery_store_b_df.iterrows():
        score = fuzz.partial_ratio(row_a['Name'].upper(), row_b['Name'].upper())
        if score > best_score:
            best_score = score
            best_match = row_b
    return best_match, best_score

# New column for storing best match data
grocery_store_a_df['Best_Match'] = None
grocery_store_a_df['Match_Score'] = None

# Dictionary to track rows from grocery_store_b_df that are already matched
matched_rows = {}

# Wrap the iteration with tqdm to show a progress bar
for idx_a, row_a in tqdm(grocery_store_a_df.iterrows(), total=grocery_store_a_df.shape[0], desc="Matching rows"):
    best_match, best_score = find_best_match(row_a, full_grocery_b_df)
    
    # Check if the row from grocery_store_b_df has already been used
    if best_match['Name'] in matched_rows:
        existing_score = matched_rows[best_match['Name']]

        if best_score < existing_score:
            # Set all values in the original row to null
            grocery_store_a_df.loc[idx_a] = [None] * len(row_a)
        else:
            # Set the new row's concatenated values to null except for the name of the b row
            grocery_store_a_df.loc[idx_a, 'Best_Match'] = None
            grocery_store_a_df.loc[idx_a, 'Match_Score'] = None
    else:
        # Store the best match and score in the matched rows dictionary
        matched_rows[best_match['Name']] = best_score
        # Add the best match and score to grocery_store_a_df
        grocery_store_a_df.loc[idx_a, 'Best_Match'] = best_match['Name']
        grocery_store_a_df.loc[idx_a, 'Match_Score'] = best_score

# Final DataFrame will contain the best match and score for each row in grocery_store_a_df
print(grocery_store_a_df)


Matching rows:   0%|          | 14/4935 [00:04<24:12,  3.39it/s]


KeyboardInterrupt: 

In [56]:
import gensim.downloader as api
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pretrained Word2Vec model
model = api.load("word2vec-google-news-300")

def get_sentence_vector(sentence, model):
    words = sentence.lower().split()  # Tokenizing the sentence
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # Return zero vector if no words found
    return np.mean(word_vectors, axis=0)

# Example sentences
sentence1 = "I love programming"
sentence2 = "I enjoy coding"

# Get vector representations
vec1 = get_sentence_vector(sentence1, model)
vec2 = get_sentence_vector(sentence2, model)

# Compute cosine similarity
similarity = cosine_similarity([vec1], [vec2])[0][0]
print(f"Cosine similarity: {similarity}")


ModuleNotFoundError: No module named 'gensim'

In [36]:
# Print the row where the 'Name' column is equal to 'PAN PASAS'
print(full_grocery_b_df[full_grocery_b_df["Name"] == "PAN PASAS"])


           Name Price Amount  Quantity is_match  Amount_Value Unit UnitPrice
2926  PAN PASAS  3.65  16 OZ         1      NaN          16.0   OZ  0.228125


In [178]:
grocery_store_a_df.to_csv("grocery_store_a_df.csv", index=False)

In [37]:
# CONVERT UNITS
full_grocery_b_df["Unit"] = full_grocery_b_df["Unit"].str.replace(" ", "", regex=True)
# Get all unique values in the 'Unit' column
unique_units = full_grocery_b_df['Unit'].unique()

# Print the unique values
print(unique_units)

['OZ' 'LB' 'RO' 'CT' 'FT' 'EA' 'LT' 'DZ' 'SB' 'PK' 'GRM' 'ML' 'QT' 'CP'
 'GL' 'YD' 'PCS' 'TB' 'FO' 'CM' 'PG' 'IN' 'UN' 'GC']


In [38]:
import pandas as pd
grocery_store_a_df = pd.read_csv("grocery_store_a_df.csv")
# Mapping dictionary from Store A to Store B
unit_mapping = {
    'Ounces': 'OZ',
    'Oz': 'OZ',
    'Fl oz': 'FO',
    None: None,  # Keep None as None
    'G': 'GRM',
    '100 ct': 'CT',
    'Fluid ounces': 'FO',
    'Sq ft': 'FT',
    'Ea': 'EA',
    'Count': 'CT',
    'Lb': 'LB',
    'Per 100 count': 'CT',
    'Ml': 'ML',
    'Ft': 'FT'
}

def map_units(df, unit_col="Unit", unit_price_col="UnitPrice"):
    """
    Maps units and adjusts unit price for count-based units.

    Parameters:
        df (pd.DataFrame): DataFrame with unit and unit price columns.
        unit_col (str): Column containing units.
        unit_price_col (str): Column containing unit price.

    Returns:
        pd.DataFrame: Updated DataFrame with mapped units and adjusted unit prices.
    """
    df[unit_col] = df[unit_col].map(unit_mapping).fillna(df[unit_col])  # Map units

    # Adjust UnitPrice for count-based units (CT)
    count_mask = df[unit_col] == 'CT'
    df.loc[count_mask, unit_price_col] = df.loc[count_mask, unit_price_col] / 100

    return df

# Example usage
grocery_store_a_df = map_units(grocery_store_a_df)
print(grocery_store_a_df["Unit"].unique())

['OZ' 'FO' nan 'GRM' 'CT' 'FT' 'EA' 'LB' 'ML']


In [39]:
import pandas as pd

def merge_best_matches(df_a, df_b, best_match_col="Best_Match", key_col="Name"):
    """
    Merges rows from df_b into df_a based on the best match column.

    Parameters:
        df_a (pd.DataFrame): The DataFrame containing the best match column.
        df_b (pd.DataFrame): The DataFrame with full data to merge.
        best_match_col (str): The column in df_a containing best match names.
        key_col (str): The column in df_b to match with best_match_col.

    Returns:
        pd.DataFrame: A new DataFrame with merged rows from df_b where matches exist.
    """
    # Perform a left join, keeping only rows where Best_Match is not None
    merged_df = df_a.merge(df_b, left_on=best_match_col, right_on=key_col, how="left")

    return merged_df

# Example usage
full_df = merge_best_matches(grocery_store_a_df, full_grocery_b_df)
print(full_df)

                                                 Name_x  Price_x  UnitPrice_x  \
0     Pan Pepin, Finest Hawaiian Burger, 12 oz, 8 Count     2.68        0.223   
1       Dove Gentle Exfoliating Beauty Bar, 4 oz, 6 Bar     7.47        0.332   
2     Line 39 Cabernet Sauvignon Red Wine, Californi...    12.98        0.511   
3              Pan Pepin, Hot Dog Buns, 10ct, 15 Ounces     2.74        0.183   
4     Borden Spreadable Butter with Canola Oil, 8 oz...     3.97        0.496   
...                                                 ...      ...          ...   
4979                                                NaN      NaN          NaN   
4980  Orville Redenbacher's Popping & Topping Butter...    28.02          NaN   
4981  Heat Lamp Bulb For Reptile 50w 110v Tortoise W...     3.59          NaN   
4982  School Zone Beginning Reading Grades 1-2 Workb...    10.19          NaN   
4983  Shamrock Gumball Machine St. Patrick's Day Rec...    12.99          NaN   

     UnitPriceString Unit_x

In [40]:
# Go through each row of full df
# If the Units from unit x and unit y are the same, then do abs(unit x - unit y) as the unit difference otherwise none
# have the price difference be the absolute value of price x - price y
full_df["Unit_Difference"] = full_df.apply(
    lambda row: abs(row["UnitPrice_x"] - row["UnitPrice_y"]) 
    if row["Unit_x"] == row["Unit_y"] else None, 
    axis=1
)

# Compute price difference
full_df["Price_Difference"] = abs(full_df["Price_x"] - full_df["Price_y"])
print(full_df)

                                                 Name_x  Price_x  UnitPrice_x  \
0     Pan Pepin, Finest Hawaiian Burger, 12 oz, 8 Count     2.68        0.223   
1       Dove Gentle Exfoliating Beauty Bar, 4 oz, 6 Bar     7.47        0.332   
2     Line 39 Cabernet Sauvignon Red Wine, Californi...    12.98        0.511   
3              Pan Pepin, Hot Dog Buns, 10ct, 15 Ounces     2.74        0.183   
4     Borden Spreadable Butter with Canola Oil, 8 oz...     3.97        0.496   
...                                                 ...      ...          ...   
4979                                                NaN      NaN          NaN   
4980  Orville Redenbacher's Popping & Topping Butter...    28.02          NaN   
4981  Heat Lamp Bulb For Reptile 50w 110v Tortoise W...     3.59          NaN   
4982  School Zone Beginning Reading Grades 1-2 Workb...    10.19          NaN   
4983  Shamrock Gumball Machine St. Patrick's Day Rec...    12.99          NaN   

     UnitPriceString Unit_x

In [41]:
full_df.to_csv("full_df.csv", index=False)

In [ ]:
sub_df = full_df[full_df["Match_Score"] > MATCH_SCORE_THRESHOLD]

# Display the sub_df to verify
print(sub_df.shape)

# Sort by Price_Difference (ascending order)
sorted_by_price = sub_df.sort_values(by="Price_Difference", ascending=False)

# Sort by Unit_Difference (ascending order)
sorted_by_unit = sub_df.sort_values(by="Unit_Difference", ascending=False)

sorted_by_price.to_csv("sorted_by_price.csv", index=False)
sorted_by_unit.to_csv("sorted_by_unit.csv", index=False)

# Optionally, print confirmation
print("Sorted DataFrames saved successfully.")


(43, 17)
Sorted by Unit Difference:
899     0.199269
424     0.179125
866     0.093000
382     0.070500
4783    0.064407
900     0.051182
1287    0.044635
423     0.043000
198     0.041124
2306    0.033595
850     0.030704
902     0.027576
851     0.022565
199     0.017708
1153    0.017571
939     0.017333
1151    0.003258
44      0.000571
1152    0.000310
15           NaN
161          NaN
162          NaN
938          NaN
1231         NaN
1232         NaN
1370         NaN
1773         NaN
1956         NaN
2032         NaN
2331         NaN
2807         NaN
3265         NaN
3346         NaN
3459         NaN
3639         NaN
3923         NaN
3924         NaN
4060         NaN
4334         NaN
4379         NaN
4427         NaN
4624         NaN
4814         NaN
Name: Unit_Difference, dtype: float64
